In [4]:
from playwright.async_api import async_playwright
import nest_asyncio
nest_asyncio.apply()

In [18]:
async def scrape_tweet(url: str)->dict:
    _xhr_calls = []

    async with async_playwright() as pw:

        async def intercept_response(response):
            if response.request.resource_type == "xhr":
                _xhr_calls.append(response)

        browser = await pw.chromium.launch(headless=True)
        #context =  browser.new_context(viewport={"width":1920, "height":1000})
        page = await browser.new_page()
        
        await page.goto(url)
        # enable background request intercepting:
        page.on("response", intercept_response)
        # go to url and wait for the page to load
        
        await page.wait_for_selector("[data-testid='tweet']")

        # find all tweet background requests:
        tweet_calls = [f for f in _xhr_calls if "TweetResultByRestId" in f.url]
        for xhr in tweet_calls:
            data = await xhr.json()
            return data["data"]["tweetResult"]["result"]

In [19]:
print(await scrape_tweet("https://twitter.com/Scrapfly_dev/status/1664267318053179398"))

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


TypeError: 'coroutine' object is not subscriptable